In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [123]:
usersDataTrain = pd.read_csv('../setTraining.csv')

In [121]:
usersDataTrain.head()

,Unnamed: 0,person,event_checkout,event_conversion,event_viewed product,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,condition_Novo,...,Parte del dia_Tarde,marca_apple,marca_asus,marca_lenovo,marca_lg,marca_motorola,marca_quantum,marca_samsung,marca_sony,label
0,0,0008ed71,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,...,2.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
1,1,000c79fe,1.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,001802e4,1.0,0.0,4.0,3.0,0.0,2.0,0.0,0.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,0019e639,15.0,2.0,189.0,118.0,0.0,26.0,62.0,0.0,...,97.0,3.0,0.0,0.0,1.0,138.0,0.0,64.0,0.0,0
4,4,001ca5ee,1.0,0.0,52.0,9.0,0.0,36.0,8.0,0.0,...,4.0,53.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [122]:
usersDataTrain.columns

Index(['Unnamed: 0', 'person', 'event_checkout', 'event_conversion',
       'event_viewed product', 'condition_Bom', 'condition_Bom - Sem Touch ID',
       'condition_Excelente', 'condition_Muito Bom', 'condition_Novo',
       'storage_128GB', 'storage_16GB', 'storage_256GB', 'storage_32GB',
       'storage_4GB', 'storage_512MB', 'storage_64GB', 'storage_8GB',
       'device_type_Computer', 'device_type_Smartphone', 'device_type_Tablet',
       'device_type_Unknown', 'Parte del dia_Madrugada',
       'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde',
       'marca_apple', 'marca_asus', 'marca_lenovo', 'marca_lg',
       'marca_motorola', 'marca_quantum', 'marca_samsung', 'marca_sony',
       'label'],
      dtype='object')

In [82]:
usersDataTrain_encoded= usersDataTrain.drop( ['person','Unnamed: 0','Parte del dia_Madrugada',
       'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde'],axis=1)

In [83]:
total_train =usersDataTrain_encoded

In [88]:
users1s = usersDataTrain_encoded[usersDataTrain_encoded["label"] == 1]

total_train = pd.concat([total_train,users1s], ignore_index=True)
total_train= pd.concat([total_train,users1s], ignore_index=True)
total_train = pd.concat([total_train,users1s], ignore_index=True)
total_train = pd.concat([total_train,users1s], ignore_index=True)
total_train["label"].value_counts()

0    16859
1    14805
Name: label, dtype: int64

In [89]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_train.drop(['label'], axis=1).values, total_train["label"] , test_size=0.20, random_state=75)


In [90]:
sm = SMOTE(random_state=12, ratio = 1.0)
training_target = usersDataTrain_encoded['label']
x_res, y_res = sm.fit_sample(X_train, y_train)
print(y_train.value_counts(), np.bincount(y_res))

/anaconda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:77: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


0    13534
1    11797
Name: label, dtype: int64 [13534 13534]


In [35]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 7)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [37]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Create a random forest Classifier. By convention, clf means 'Classifier'

clf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 2 folds for each of 100 candidates, totalling 200 fits
[CV] n_estimators=288, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=43, bootstrap=True 
[CV] n_estimators=288, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=43, bootstrap=True 
[CV] n_estimators=466, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False 
[CV] n_estimators=466, min_samples_split=5, min_samples_leaf=1, max_features=auto, max_depth=10, bootstrap=False 
[CV]  n_estimators=288, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=43, bootstrap=True, total=   2.9s
[CV] n_estimators=822, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=93, bootstrap=False 
[CV]  n_estimators=288, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=43, bootstrap=True, total=   3.1s
[CV] n_estimators=822, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=93, bootstrap=Fa

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.5min


[CV]  n_estimators=911, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=43, bootstrap=True, total=  15.9s
[CV] n_estimators=288, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=26, bootstrap=True 
[CV]  n_estimators=288, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=26, bootstrap=True, total=   5.3s
[CV] n_estimators=911, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=76, bootstrap=True 
[CV]  n_estimators=288, min_samples_split=2, min_samples_leaf=4, max_features=sqrt, max_depth=26, bootstrap=True, total=   4.9s
[CV] n_estimators=911, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=76, bootstrap=True 
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=26, bootstrap=True, total=  16.6s
[CV]  n_estimators=1000, min_samples_split=10, min_samples_leaf=2, max_features=auto, max_depth=26, bootstrap=True, total=  17.5s
[CV] n_estimato

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  7.6min


[CV]  n_estimators=733, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=26, bootstrap=False, total=  13.5s
[CV]  n_estimators=644, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=76, bootstrap=True, total=  10.5s
[CV] n_estimators=733, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=43, bootstrap=True 
[CV] n_estimators=733, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=43, bootstrap=True 
[CV]  n_estimators=644, min_samples_split=2, min_samples_leaf=2, max_features=auto, max_depth=76, bootstrap=True, total=  11.0s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=733, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=26, bootstrap=False, total=  13.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=None, bootstrap=False 
[CV]  n_estimators=200, mi

[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 10.2min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [38]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 43,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 288}

In [91]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(bootstrap= True,
 max_depth= 43,
 max_features =  'sqrt',
 min_samples_leaf= 1,
 min_samples_split= 10,
 n_estimators= 288)

In [92]:
clf1.fit(x_res, y_res)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=43, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=288, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [93]:
result = clf1.predict_proba(X_test)

In [94]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [95]:
from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.999079647256439

In [96]:
clf1.score(X_test, y_test)

0.9902100110532134

In [97]:
test = pd.read_csv('../setTest2.csv')

In [98]:
person = test["person"]

In [99]:
test.head()

,Unnamed: 0,person,event_checkout,event_conversion,event_viewed product,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,condition_Novo,...,Parte del dia_Noche,Parte del dia_Tarde,marca_apple,marca_asus,marca_lenovo,marca_lg,marca_motorola,marca_quantum,marca_samsung,marca_sony
0,0,00091926,2.0,0.0,250.0,60.0,0.0,75.0,117.0,0.0,...,143.0,0.0,220.0,0.0,0.0,0.0,14.0,0.0,18.0,0.0
1,1,000ba417,6.0,1.0,153.0,115.0,0.0,11.0,34.0,0.0,...,31.0,93.0,8.0,0.0,0.0,4.0,37.0,0.0,110.0,1.0
2,2,000e4d9e,1.0,0.0,339.0,124.0,0.0,53.0,163.0,0.0,...,198.0,123.0,21.0,0.0,0.0,1.0,2.0,0.0,301.0,15.0
3,3,000e619d,1.0,0.0,28.0,14.0,0.0,7.0,8.0,0.0,...,26.0,3.0,7.0,0.0,0.0,3.0,7.0,0.0,12.0,0.0
4,4,001001be,3.0,1.0,41.0,16.0,9.0,1.0,19.0,0.0,...,45.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
test = test.drop(['person','Unnamed: 0'],axis=1)

In [101]:
test.head()

,event_checkout,event_conversion,event_viewed product,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,condition_Novo,storage_128GB,storage_16GB,...,Parte del dia_Noche,Parte del dia_Tarde,marca_apple,marca_asus,marca_lenovo,marca_lg,marca_motorola,marca_quantum,marca_samsung,marca_sony
0,2.0,0.0,250.0,60.0,0.0,75.0,117.0,0.0,41.0,86.0,...,143.0,0.0,220.0,0.0,0.0,0.0,14.0,0.0,18.0,0.0
1,6.0,1.0,153.0,115.0,0.0,11.0,34.0,0.0,0.0,115.0,...,31.0,93.0,8.0,0.0,0.0,4.0,37.0,0.0,110.0,1.0
2,1.0,0.0,339.0,124.0,0.0,53.0,163.0,0.0,1.0,108.0,...,198.0,123.0,21.0,0.0,0.0,1.0,2.0,0.0,301.0,15.0
3,1.0,0.0,28.0,14.0,0.0,7.0,8.0,0.0,0.0,8.0,...,26.0,3.0,7.0,0.0,0.0,3.0,7.0,0.0,12.0,0.0
4,3.0,1.0,41.0,16.0,9.0,1.0,19.0,0.0,0.0,39.0,...,45.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
test.shape

(17532, 32)

In [103]:
nombreCol = []
for namecolumn in usersDataTrain.columns:
     if ( not namecolumn in test.columns):
            nombreCol.append(namecolumn)
print(nombreCol)
for namecolumn in test.columns:
     if ( not namecolumn in usersDataTrain.columns):
            nombreCol.append(namecolumn)
print(nombreCol)

['Unnamed: 0', 'person', 'label']
['Unnamed: 0', 'person', 'label']


In [104]:
result = clf1.predict_proba(test)

In [105]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [106]:
person = person.to_frame()

In [107]:
person['label'] = result2

In [108]:
person.head()

,person,label
0,00091926,0.059028
1,000ba417,0.104167
2,000e4d9e,0.110764
3,000e619d,0.038194
4,001001be,0.027546


In [109]:
(person["label"]<0.5).value_counts()

True     17145
False      387
Name: label, dtype: int64

In [110]:
kaggle = pd.read_csv('../data/trocafone_kaggle_test.csv')

In [111]:
kaggle.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [112]:
resultado = pd.merge(kaggle,person,on="person",how = "left")

In [113]:
resultado.head()

,person,label
0,4886f805,0.000000
1,0297fc1e,0.093750
2,2d681dd8,0.027778
3,cccea85e,0.150568
4,4c8a8b93,0.100694


In [114]:
resultado["label"].isnull().value_counts()

False    17532
True      1883
Name: label, dtype: int64

In [115]:
resultado["label"] = resultado["label"].fillna(0)

In [116]:
resultado["label"].isnull().value_counts()

False    19415
Name: label, dtype: int64

In [117]:
(resultado["label"]<0.5).value_counts()

True     19028
False      387
Name: label, dtype: int64

In [118]:
resultado.to_csv('resultadoRandomForest4.csv', index=False)